In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable

In [2]:
trYs = torch.tensor(torch.load("../TrainData_traits.pt"),dtype=torch.float32)
trYs = trYs.reshape(trYs.shape[0],3)

trXgenos = torch.tensor(torch.load("../TrainData_genos.pt"),dtype=torch.float32)
trXgenos = trXgenos.reshape(trXgenos.shape[0],trXgenos.shape[1])
print(trYs.shape)
print(trXgenos.shape)

D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
D:\ANACONDA\lib\site-packages\ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


torch.Size([2000, 3])
torch.Size([2000, 26086])


In [3]:
class Mydataset(Dataset):
    
    def __init__(self,trYs,trXgenos,transform=None):
        self.traits = trYs
        self.trXgenos = trXgenos
        
        assert len(self.traits) == len(self.trXgenos), "samples not match"
        
        self.transform = transform
        
        self.sample_set = []
        for idx in range(len(self.trXgenos)):
            self.sample_set.append((self.trXgenos[idx],self.traits[idx]))
            
    def __getitem__(self, index):
        genes,traits = self.sample_set[index]
        if self.transform is not None:
            genes = self.transform(genes)
            
        return genes,traits
    
    def __len__(self):
        return len(self.trXgenos)
    
trainData = Mydataset(trYs,trXgenos)
testData = Mydataset(trYs,trXgenos)

In [4]:
batch_size = 32


trainLoader = DataLoader(dataset=trainData,batch_size=batch_size,shuffle=True)
testLoader = DataLoader(dataset=trainData,batch_size=batch_size,shuffle=False)


In [5]:
"""
class AlexNet(nn.Module):
    def __init__(self,init_weights=False):
        super(AlexNet, self).__init__()


        '''
        
        data size:3channel x32x32
        1st convolutional layer, input channel=3, output channel=96, kernel size=3x3, step=1*1'''

        '''feature map size=(32-3)/1+1=30,feature map dimension 96*30*30'''

        self.features = nn.Sequential(
            nn.Conv1d(26086,10000, kernel_size=3, stride=1), # output channel: pic size;
            nn.ReLU(inplace=True),
            #nn.BatchNorm1d(96), # normalization

            nn.MaxPool1d(kernel_size=3, stride=2),  # kernel 3x3; step 2x2 -> (30-3)/2 + 1 = 14x14

            nn.Conv1d(10000, 2000, kernel_size=(3,3), stride=(1,1)), # (14-3)/1 + 1 = 12 x 12
            nn.ReLU(inplace=True),
            #nn.BatchNorm1d(256),

            nn.MaxPool1d(kernel_size=3, stride=2),  # (12 - 3) /2 +1 = 5x5

            nn.Conv1d(2000, 40, kernel_size=3, padding=1, stride=1),  # (5-3+2*1)/1+1=5x5
            
        )

        self.classifier = nn.Sequential(

            nn.Linear(40*14, 120),

            nn.Dropout(0.5),

            nn.Linear(120, 84),

            nn.Dropout(0.5),

            nn.Linear(84, 1)
        )
        if init_weights:
            self._initialize_weights()


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')  
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = x.unsqueeze(1)
        
        out = self.features(x)

        out = torch.flatten(out,start_dim=1)

        #out = out.reshape(-1, 256 * 2 * 2)

        out = self.classifier(out)

        return out
    
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(dataloader): # enumerate

            images=Variable(images)
            labels=Variable(labels)
            #print(labels)
            # Forward pass
            #outputs = model(images)
            #loss = criterion(outputs, labels)

            X, y = images.to(device), labels.to(device)
            pred = model(X)
            predict = torch.max(pred,dim=1)[1]
            #print("##################")
            #print(predict)
            test_loss += loss_fn(pred, y).item()
            correct += (predict == y).sum().item()
            #correct += (pred == y).type(torch.max).sum().item()
            size += len(labels)
        correct /= size
        test_loss /= num_batches

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
"""


In [8]:
from AlexNet import *
import gc

gc.collect()

torch.cuda.empty_cache()
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = AlexNet().to(device)

#summary.summary(model, input_size=(3, 32, 32), batch_size=128, device="cpu")

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

total_step = len(trainLoader)
epochs = 30
for epoch in range(epochs):
    model.train()
    i=0
    for i,(images, labels) in enumerate(trainLoader):
        optimizer.zero_grad()
        images=Variable(images).to(device)
        labels=Variable(labels).to(device)
        # Forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)

        # Backward and optimize
        #optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, epochs, i + 1, total_step, loss.item()))
        i += 1
    #print("temp train results: ")
    #print(torch.max(outputs, dim=1)[1])
    #print(labels)
    model.eval()
    print("test")
    test(testLoader,model,criterion)


Using cuda device


RuntimeError: CUDA out of memory. Tried to allocate 2.92 GiB (GPU 0; 6.00 GiB total capacity; 3.59 GiB already allocated; 517.95 MiB free; 3.61 GiB reserved in total by PyTorch)